<a href="https://colab.research.google.com/github/jeffwng5177/ECE1512-2023F-ProjectRepo-JeffreyWang/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project B MHIST

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("/content/gdrive/My Drive/")

Mounted at /content/gdrive


In [ ]:

import numpy as np

import os
import pandas as pd
import tensorflow as tf
import keras

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Flatten
from keras import regularizers
import keras.backend as K
from keras.models import load_model
from tensorflow.core.util import event_pb2
from tensorflow.python.lib.io import tf_record
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory


Generate Test/Train batch

In [ ]:
train_dir = '/content/gdrive/My Drive/mhist_dataset/images/train' #you should change to your directory
test_dir = '/content/gdrive/My Drive/mhist_dataset/images/test' #you should change to your directory
#train_dir = 'E:/UofTCourse/ECE1512/Assignment3/Project_B_Supp/Project_B_Supp/mhist_dataset/images/train' #you should change to your directory
#test_dir = 'E:/UofTCourse/ECE1512/Assignment3/Project_B_Supp/Project_B_Supp/mhist_dataset/images/test' #you should change to your directory


train_datagen = ImageDataGenerator(rescale=1/255.)

test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
class_mode='categorical',
interpolation='bilinear',
target_size=(224, 224),
batch_size=32,
shuffle=True)

test_generator = test_datagen.flow_from_directory(test_dir,
class_mode='categorical',
interpolation='bilinear',
target_size=(224, 224),
batch_size=32,
#shuffle=False)
shuffle=True)
print(test_generator)
print(train_generator)

Found 2192 images belonging to 2 classes.
Found 986 images belonging to 2 classes.


In [ ]:
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
BATCH_SIZE = 32
NUM_EPOCHS = 10
NUM_CLASSES = 2
TRAIN_BATCHES_NUM = int(np.floor(2176/BATCH_SIZE))
TEST_BATCHES_NUM  = int(np.floor(976/BATCH_SIZE))

Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Build and train teacher model

Teacher Model

In [ ]:
base_model_teacher = ResNet50V2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_teacher.trainable = False

prediction_layer = tf.keras.layers.Dense(2)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

inputs_t = tf.keras.Input(shape=(224, 224, 3))
x_t = data_augmentation(inputs_t)
x_t = tf.keras.applications.resnet_v2.preprocess_input(x_t)
x_t = base_model_teacher(x_t)
x_t = global_average_layer(x_t)
outputs_t = prediction_layer(x_t)
teacher_model = tf.keras.Model(inputs_t, outputs_t)
teacher_model.summary()




94683136/94668760 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                              

Teacher loss function

In [ ]:
@tf.function
def compute_teacher_loss(images, labels):
  """Compute subclass knowledge distillation teacher loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  subclass_logits = teacher_model(images, training=True)
  #print('a')
  #print(subclass_logits.shape)
  #print(labels.shape)
  # Compute cross-entropy loss for subclasses.

  # your code start from here for step 3
  cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  cross_entropy_loss_value = cce(labels, subclass_logits)
  #print(cross_entropy_loss_value)
  return cross_entropy_loss_value



Train and evaluation function

In [ ]:

def compute_num_correct(model, images, labels):
  """Compute number of correctly classified images in a batch.

  Args:
    model: Instance of tf.keras.Model.
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Number of correctly classified images.
  """
  class_logits = model(images, training=False)
  return tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)


def train_and_evaluate(model, compute_loss_fn, epochs, lr):
  """Perform training and evaluation for a given model.

  Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
      images, and labels.
  """

  # your code start from here for step 4
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  for epoch in range(1, epochs + 1):
    # Run training.
    print('Epoch {}: '.format(epoch), end='')

    for batch in range(TRAIN_BATCHES_NUM):
      images, labels = train_generator.next()

      with tf.GradientTape() as tape:
         # your code start from here for step 4
        loss_value = compute_loss_fn(images, labels)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))
      #print(model.trainable_variables)
      #print("grads")
      #print(np.array(grads).shape)
      #print("model.trainable_variables")
      #print(np.array(model.trainable_variables).shape)

    # Run evaluation.
    num_correct = 0
    num_total = 0

    for batch in range(TEST_BATCHES_NUM):
      images, labels = test_generator.next()
      # your code start from here for step 4

      label_together = np.array(compute_num_correct(model, images, labels))
      label_real = label_together[2]
      label_pred = label_together[1]
      #print("label_predicted")
      #print(label_pred)
      #print("label_real")
      #print(label_real)
      sum_same = np.array((label_real == label_pred),int)

      num_total += labels.shape[0]
      num_correct += tf.reduce_sum(sum_same)
      #print('num_correct')
      #print(num_correct)
      #print('num_correct')
      #print(num_total)
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))



Feature Extraction Teacher

In [ ]:
train_and_evaluate(teacher_model, compute_teacher_loss,epochs=10,lr=0.0001)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 63.54%
Epoch 2: Class_accuracy: 63.10%
Epoch 3: Class_accuracy: 63.21%
Epoch 4: Class_accuracy: 64.05%
Epoch 5: Class_accuracy: 64.78%
Epoch 6: Class_accuracy: 62.05%
Epoch 7: Class_accuracy: 63.21%
Epoch 8: Class_accuracy: 64.47%
Epoch 9: Class_accuracy: 64.36%
Epoch 10: Class_accuracy: 62.58%


Fine Tuning Teacher

In [ ]:
base_model_teacher.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_teacher.layers))

# Fine-tune from this layer onwards
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_teacher.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  190


In [ ]:
train_and_evaluate(teacher_model, compute_teacher_loss,epochs=25,lr=1e-5)

In [ ]:
teacher_model.save("teacher_model")

INFO:tensorflow:Assets written to: teacher_model/assets


In [ ]:
#teacher_model = keras.models.load_model("teacher_model")
teacher_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0   

# Build and train Student Model

Student Model with KD

In [ ]:
base_model_student1 = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_student1.trainable = False

dense1_layer1 = tf.keras.layers.Dense(64,activation='relu')
drop_out1_layer1 = tf.keras.layers.Dropout(rate=0.2)
drop_out2_layer1 = tf.keras.layers.Dropout(rate=0.2)

prediction_layer1 = tf.keras.layers.Dense(2)

global_average_layer1 = tf.keras.layers.GlobalAveragePooling2D()

inputs_s1 = tf.keras.Input(shape=(224, 224, 3))
x_s1 = data_augmentation(inputs_s1)
x_s1 = tf.keras.applications.mobilenet_v2.preprocess_input(x_s1)
#x_s1 = tf.keras.applications.resnet_v2.preprocess_input(x_s1)
x_s1 = base_model_student1(x_s1)
x_s1 = global_average_layer1(x_s1)
x_s1 = drop_out1_layer1(x_s1)
x_s1 = dense1_layer1(x_s1)
x_s1 = drop_out2_layer1(x_s1)

outputs_s1 = prediction_layer1(x_s1)
student_model = tf.keras.Model(inputs_s1, outputs_s1)
student_model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                       

Student Model without KD

In [ ]:
base_model_student2 = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
#base_model_student2 = ResNet50V2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_student2.trainable = False

prediction_layer2 = tf.keras.layers.Dense(2)

global_average_layer2 = tf.keras.layers.GlobalAveragePooling2D()

dense1_layer2 = tf.keras.layers.Dense(64,activation='relu')
drop_out1_layer2 = tf.keras.layers.Dropout(rate=0.2)
drop_out2_layer2 = tf.keras.layers.Dropout(rate=0.2)

inputs_s2 = tf.keras.Input(shape=(224, 224, 3))
x_s2 = data_augmentation(inputs_s2)
x_s2 = tf.keras.applications.mobilenet_v2.preprocess_input(x_s2)
#x_s2 = tf.keras.applications.resnet_v2.preprocess_input(x_s2)
x_s2 = base_model_student2(x_s2)
x_s2 = global_average_layer(x_s2)

x_s2 = drop_out1_layer2(x_s2)
x_s2 = dense1_layer2(x_s2)
x_s2 = drop_out2_layer2(x_s2)

outputs_s2 = prediction_layer2(x_s2)
student_model_no_distillation = tf.keras.Model(inputs_s2, outputs_s2)
student_model_no_distillation.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                           

State of Art Student Model

In [ ]:
base_model_student3 = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_student3.trainable = False

prediction_layer3 = tf.keras.layers.Dense(2)

global_average_layer3 = tf.keras.layers.GlobalAveragePooling2D()

dense1_layer3 = tf.keras.layers.Dense(64,activation='relu')
drop_out1_layer3 = tf.keras.layers.Dropout(rate=0.2)
drop_out2_layer3 = tf.keras.layers.Dropout(rate=0.2)

inputs_s3 = tf.keras.Input(shape=(224, 224, 3))
x_s3 = data_augmentation(inputs_s3)
x_s3 = tf.keras.applications.mobilenet_v2.preprocess_input(x_s3)
x_s3 = base_model_student2(x_s3)
x_s3 = global_average_layer(x_s3)

x_s3 = drop_out1_layer2(x_s3)
x_s3 = dense1_layer2(x_s3)
x_s3 = drop_out2_layer2(x_s3)

outputs_s3 = prediction_layer2(x_s3)
student_model_improved = tf.keras.Model(inputs_s3, outputs_s3)
student_model_improved.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_3 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_3 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                           

Student loss function

In [ ]:
# Hyperparameters for distillation (need to be tuned).
ALPHA = 0.5 # task balance between cross-entropy and distillation loss
DISTILLATION_TEMPERATURE = 64. #temperature hyperparameter 1,2,4,16,32,64

def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: tf.Tensor):
  """Compute distillation loss.

  This function computes cross entropy between softened logits and softened
  targets. The resulting loss is scaled by the squared temperature so that
  the gradient magnitude remains approximately constant as the temperature is
  changed. For reference, see Hinton et al., 2014, "Distilling the knowledge in
  a neural network."

  Args:
    teacher_logits: A Tensor of logits provided by the teacher.
    student_logits: A Tensor of logits provided by the student, of the same
      shape as `teacher_logits`.
    temperature: Temperature to use for distillation.

  Returns:
    A scalar Tensor containing the distillation loss.
  """
 # your code start from here for step 3
  soft_targets = keras.backend.softmax(teacher_logits/temperature) #softmax of teacher logits

  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss(images, labels):
  """Compute subclass knowledge distillation student loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  subclass_logits = student_model(images, training=True)



  teacher_subclass_logits = teacher_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_subclass_logits, subclass_logits,
                      DISTILLATION_TEMPERATURE)

  #cross_entropy_loss_value = keras.backend.softmax(student_subclass_logits)
  cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  cross_entropy_loss_value = cce(labels, subclass_logits)
  return ALPHA * cross_entropy_loss_value + (1- ALPHA) * distillation_loss_value

Student Loss Function without KD

In [ ]:
def compute_plain_cross_entropy_loss(images, labels):

  subclass_logits = student_model_no_distillation(images, training=True)
  cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  cross_entropy_loss_value = cce(labels, subclass_logits)
  return cross_entropy_loss_value

Train and evaluation

In [ ]:

def compute_num_correct(model, images, labels):
  """Compute number of correctly classified images in a batch.

  Args:
    model: Instance of tf.keras.Model.
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Number of correctly classified images.
  """
  class_logits = model(images, training=True)
  #print('Compute num correct')
  #tf.print(class_logits)
  #tf.print(tf.argmax(class_logits, -1))
  #tf.print(tf.argmax(labels, -1))
  return tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)


def train_and_evaluate(model, compute_loss_fn, epochs, lr):
  """Perform training and evaluation for a given model.

  Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
      images, and labels.
  """

  # your code start from here for step 4
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  for epoch in range(1, epochs + 1):
    # Run training.
    print('Epoch {}: '.format(epoch), end='')

    for batch in range(TRAIN_BATCHES_NUM):
      images, labels = train_generator.next()

      with tf.GradientTape() as tape:
         # your code start from here for step 4
        loss_value = compute_loss_fn(images, labels)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))
      #print(model.trainable_variables)
      #print("grads")
      #print(np.array(grads).shape)
      #print("model.trainable_variables")
      #print(np.array(model.trainable_variables).shape)

    # Run evaluation.
    num_correct = 0
    num_total = 0

    for batch in range(TEST_BATCHES_NUM):
      images, labels = test_generator.next()
      # your code start from here for step 4

      label_together = np.array(compute_num_correct(model, images, labels))
      label_real = label_together[2]
      label_pred = label_together[1]
      #print("label_predicted")
      #print(label_pred)
      #print("label_real")
      #print(label_real)
      sum_same = np.array((label_real == label_pred),int)

      num_total += labels.shape[0]
      #num_correct += tf.reduce_sum(sum_same)
      num_correct += label_together[0]
      #print('num_correct')
      #print(num_correct)
      #print('num_correct')
      #print(num_total)
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))



Feature Extraction Student without KD

In [ ]:
base_model_student2.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_teacher.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_student2.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  190


In [ ]:
train_and_evaluate(student_model_no_distillation, compute_plain_cross_entropy_loss,epochs=10,lr=0.001)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 65.30%
Epoch 2: Class_accuracy: 63.84%
Epoch 3: Class_accuracy: 62.58%
Epoch 4: Class_accuracy: 66.67%
Epoch 5: Class_accuracy: 71.80%
Epoch 6: Class_accuracy: 67.19%
Epoch 7: Class_accuracy: 69.29%
Epoch 8: Class_accuracy: 68.66%
Epoch 9: Class_accuracy: 69.08%
Epoch 10: Class_accuracy: 71.17%


Fine tuning Student (without KD)

In [ ]:
base_model_student2.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_teacher.layers))

# Fine-tune from this layer onwards
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_student2.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  190


Train Student (without KD)

In [ ]:
train_and_evaluate(student_model_no_distillation, compute_plain_cross_entropy_loss,epochs=25,lr=1e-4)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 68.45%
Epoch 2: Class_accuracy: 72.22%
Epoch 3: Class_accuracy: 77.57%
Epoch 4: Class_accuracy: 80.71%
Epoch 5: Class_accuracy: 80.71%
Epoch 6: Class_accuracy: 80.50%
Epoch 7: Class_accuracy: 80.71%
Epoch 8: Class_accuracy: 80.19%
Epoch 9: Class_accuracy: 81.66%
Epoch 10: Class_accuracy: 82.18%
Epoch 11: Class_accuracy: 83.44%
Epoch 12: Class_accuracy: 82.91%
Epoch 13: Class_accuracy: 82.18%
Epoch 14: Class_accuracy: 81.55%
Epoch 15: Class_accuracy: 81.56%
Epoch 16: Class_accuracy: 82.81%
Epoch 17: Class_accuracy: 81.76%
Epoch 18: Class_accuracy: 81.87%
Epoch 19: Class_accuracy: 81.34%
Epoch 20: Class_accuracy: 82.49%
Epoch 21: Class_accuracy: 82.70%
Epoch 22: Class_accuracy: 81.87%
Epoch 23: Class_accuracy: 81.24%
Epoch 24: Class_accuracy: 84.17%
Epoch 25: Class_accuracy: 82.08%


In [ ]:
#student_model_no_distillation.save("student_model_no_distillation")

In [ ]:
#student_model_no_distillation = keras.models.load_model("student_model_no_distillation")
student_model_no_distillation.summary()

Feature Extraction Student with KD

In [ ]:
train_and_evaluate(student_model, compute_student_loss,epochs=10,lr=0.001)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 64.15%
Epoch 2: Class_accuracy: 62.47%
Epoch 3: Class_accuracy: 63.42%
Epoch 4: Class_accuracy: 63.52%
Epoch 5: Class_accuracy: 63.31%
Epoch 6: Class_accuracy: 62.47%
Epoch 7: Class_accuracy: 62.89%
Epoch 8: Class_accuracy: 65.62%
Epoch 9: Class_accuracy: 63.52%
Epoch 10: Class_accuracy: 62.16%


Fine Tune Student (with KD)

In [ ]:
base_model_student1.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_teacher.layers))

# Fine-tune from this layer onwards
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_student1.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  190


In [ ]:
train_and_evaluate(student_model, compute_student_loss,epochs=25,lr=1e-4)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 73.69%
Epoch 2: Class_accuracy: 77.15%
Epoch 3: Class_accuracy: 74.63%
Epoch 4: Class_accuracy: 77.15%
Epoch 5: Class_accuracy: 78.72%
Epoch 6: Class_accuracy: 78.83%
Epoch 7: Class_accuracy: 79.98%
Epoch 8: Class_accuracy: 83.23%
Epoch 9: Class_accuracy: 81.55%
Epoch 10: Class_accuracy: 79.14%
Epoch 11: Class_accuracy: 80.73%
Epoch 12: Class_accuracy: 82.60%
Epoch 13: Class_accuracy: 85.12%
Epoch 14: Class_accuracy: 83.44%
Epoch 15: Class_accuracy: 82.60%
Epoch 16: Class_accuracy: 83.23%
Epoch 17: Class_accuracy: 83.65%
Epoch 18: Class_accuracy: 83.75%
Epoch 19: Class_accuracy: 82.49%
Epoch 20: Class_accuracy: 83.65%
Epoch 21: Class_accuracy: 82.81%
Epoch 22: Class_accuracy: 82.91%
Epoch 23: Class_accuracy: 83.86%
Epoch 24: Class_accuracy: 84.38%
Epoch 25: Class_accuracy: 82.18%


In [ ]:
#student_model.save("student_model")

INFO:tensorflow:Assets written to: student_model/assets


In [ ]:
#student_model = keras.models.load_model("student_model")

FLOPs

In [ ]:
# your code start from here for step 8
!pip install keras_flops
from keras_flops import get_flops
flops = get_flops(teacher_model, batch_size=32)
print('teacher model')
print(flops)
flops = get_flops(student_model, batch_size=32)
print('student model')
print(flops)

     |████████████████████████████████| 462 kB 7.9 MB/s 
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
teacher model
223633626304
student model
19618953216


AUC teacher

In [ ]:
auc = tf.keras.metrics.AUC(from_logits=True)
auc.reset_state()

for batch in range(TEST_BATCHES_NUM):
  images, labels = test_generator.next()
  # your code start from here for step 4

  label_together = np.array(compute_num_correct(teacher_model, images, labels))
  label_real = np.array(label_together[2],float)
  label_pred = np.array(label_together[1],float)
  auc.update_state(label_real,label_pred)

print("auc")
print(auc.result().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


auc
0.80207914


AUC Student no distillation

In [ ]:

auc = tf.keras.metrics.AUC(from_logits=True)
auc.reset_state()


for batch in range(TEST_BATCHES_NUM):
  images, labels = test_generator.next()
  # your code start from here for step 4
  label_together = np.array(compute_num_correct(student_model_no_distillation, images, labels))
  #print(compute_num_correct(student_model_no_distillation, images, labels))
  label_real = np.array(label_together[2],float)
  #print(label_real)
  label_pred = np.array(label_together[1],float)
  #print(label_pred)
  auc.update_state(label_real,label_pred)
print("auc")
print(auc.result().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':


auc
0.77684546


AUC Student with distillation

In [ ]:

auc = tf.keras.metrics.AUC(from_logits=True)
auc.reset_state()


for batch in range(TEST_BATCHES_NUM):
  images, labels = test_generator.next()
  # your code start from here for step 4

  label_together = np.array(compute_num_correct(student_model, images, labels))
  label_real = np.array(label_together[2],float)
  label_pred = np.array(label_together[1],float)
  auc.update_state(label_real,label_pred)
print("auc")
print(auc.result().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


auc
0.8104153


State of Art KD

In [ ]:
from tensorflow.python.ops.math_ops import reduce_sum
import torch
import torch.nn as nn
import math

# Hyperparameters for distillation (need to be tuned).
ALPHA = 0.5 # task balance between cross-entropy and distillation loss
DISTILLATION_TEMPERATURE = 0.1 #temperature hyperparameter #2
batch_size = 32

def distillation_loss_improved(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: tf.Tensor):

  N = teacher_logits.shape[0]
  #mask for correlated samples
  mask = torch.ones((2*N, 2*N), dtype=bool)
  mask.fill_diagonal_(0)
  for i in range(N):
      mask[i, N+i] = 0
      mask[N+i, i] = 0

  z = tf.concat([student_logits,teacher_logits],0)
  z = np.array(z)
  a1 = np.zeros([2*N,2*N,teacher_logits.shape[1]])
  for i in range(2*N):
    a1[i,:,:] = z
  a2 = np.zeros([2*N,2*N,teacher_logits.shape[1]])
  for i in range(2*N):
    a2[:,i,:] = z
  sim = tf.keras.losses.cosine_similarity(a1, a2,axis=2)
  sim = np.array(sim)
  sim = abs(sim)

  negative_samples = np.array(sim[mask]).reshape(4*N*(N-1),1)
  negative_samples = -negative_samples/temperature
  negative_samples = 1 - 1/(tf.exp(negative_samples)+1)
  negative_samples = tf.math.log(negative_samples)
  negative_loss = tf.reduce_mean(negative_samples)
  negative_loss = tf.cast(negative_loss, tf.float32)

  positive_samples = tf.keras.losses.cosine_similarity(teacher_logits,student_logits)
  positive_samples = -positive_samples/temperature
  positive_samples = 1/(tf.exp(positive_samples)+1)
  positive_samples = tf.math.log(positive_samples)
  positive_loss = tf.reduce_mean(positive_samples)
  positive_loss = tf.cast(positive_loss, tf.float32)
  loss = positive_loss + negative_loss
  return loss


def compute_student_loss_new(images, labels):
  """Compute subclass knowledge distillation student loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  student_subclass_logits = student_model(images, training=True)

  # Compute subclass distillation loss between student subclass logits and
  # softened teacher subclass targets probabilities.

  # your code start from here for step 3

  teacher_subclass_logits = teacher_model(images, training=False)
  distillation_loss_value = distillation_loss_improved(teacher_subclass_logits, student_subclass_logits,
                      DISTILLATION_TEMPERATURE)
  #print("distillation_loss_value")
  #print(distillation_loss_value)

  # Compute cross-entropy loss with hard targets.

  # your code start from here for step 3

  #cross_entropy_loss_value = keras.backend.softmax(student_subclass_logits)
  cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  cross_entropy_loss_value = cce(labels, student_subclass_logits)
  #print("loss")
  #print(cross_entropy_loss_value)
  #print(distillation_loss_value)
  return ALPHA * cross_entropy_loss_value + (1- ALPHA) * distillation_loss_value



In [ ]:
base_model_student1 = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
#base_model_student1 = ResNet50V2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_student1.trainable = False

dense1_layer1 = tf.keras.layers.Dense(64,activation='relu')
drop_out1_layer1 = tf.keras.layers.Dropout(rate=0.2)
drop_out2_layer1 = tf.keras.layers.Dropout(rate=0.2)

prediction_layer1 = tf.keras.layers.Dense(2)

global_average_layer1 = tf.keras.layers.GlobalAveragePooling2D()

inputs_s1 = tf.keras.Input(shape=(224, 224, 3))
x_s1 = data_augmentation(inputs_s1)
x_s1 = tf.keras.applications.mobilenet_v2.preprocess_input(x_s1)
#x_s1 = tf.keras.applications.resnet_v2.preprocess_input(x_s1)
x_s1 = base_model_student1(x_s1)
x_s1 = global_average_layer1(x_s1)
x_s1 = drop_out1_layer1(x_s1)
x_s1 = dense1_layer1(x_s1)
x_s1 = drop_out2_layer1(x_s1)

outputs_s1 = prediction_layer1(x_s1)
student_model = tf.keras.Model(inputs_s1, outputs_s1)
student_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_7 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_7 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                           

In [ ]:
train_and_evaluate(student_model, compute_student_loss_new,epochs=10,lr=0.001)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 64.26%
Epoch 2: Class_accuracy: 64.78%
Epoch 3: Class_accuracy: 62.16%
Epoch 4: Class_accuracy: 66.14%
Epoch 5: Class_accuracy: 60.80%
Epoch 6: Class_accuracy: 64.36%
Epoch 7: Class_accuracy: 63.00%
Epoch 8: Class_accuracy: 64.15%
Epoch 9: Class_accuracy: 63.00%
Epoch 10: Class_accuracy: 62.89%


In [ ]:
base_model_student1.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_teacher.layers))

# Fine-tune from this layer onwards
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_student1.layers[:fine_tune_at]:
  layer.trainable =  False
student_model.summary()

Number of layers in the base model:  190
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_7 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_7 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                  

In [ ]:
train_and_evaluate(student_model, compute_student_loss_new,epochs=25,lr=1e-4)

Epoch 1: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Class_accuracy: 64.57%
Epoch 2: Class_accuracy: 69.71%
Epoch 3: Class_accuracy: 74.00%
Epoch 4: Class_accuracy: 71.49%
Epoch 5: Class_accuracy: 74.00%
Epoch 6: Class_accuracy: 75.89%
Epoch 7: Class_accuracy: 76.42%
Epoch 8: Class_accuracy: 77.92%
Epoch 9: Class_accuracy: 78.09%
Epoch 10: Class_accuracy: 77.36%
Epoch 11: Class_accuracy: 76.21%
Epoch 12: Class_accuracy: 77.67%
Epoch 13: Class_accuracy: 80.61%
Epoch 14: Class_accuracy: 80.50%
Epoch 15: Class_accuracy: 81.76%
Epoch 16: Class_accuracy: 78.72%
Epoch 17: Class_accuracy: 75.58%
Epoch 18: Class_accuracy: 77.57%
Epoch 19: Class_accuracy: 79.77%
Epoch 20: Class_accuracy: 78.83%
Epoch 21: Class_accuracy: 79.87%
Epoch 22: Class_accuracy: 78.93%
Epoch 23: Class_accuracy: 79.66%
Epoch 24: Class_accuracy: 78.09%
Epoch 25: Class_accuracy: 80.71%


AUC student state of art

In [ ]:

auc = tf.keras.metrics.AUC(from_logits=True)
auc.reset_state()


for batch in range(TEST_BATCHES_NUM):
  images, labels = test_generator.next()
  # your code start from here for step 4

  label_together = np.array(compute_num_correct(student_model, images, labels))
  label_real = np.array(label_together[2],float)
  label_pred = np.array(label_together[1],float)
  auc.update_state(label_real,label_pred)
print("auc")
print(auc.result().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


auc
0.7438476
